In [1]:
%pylab inline

import os,sys
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.gridspec as gridspec

from collections import defaultdict, Counter, OrderedDict

import cytograph as cg
import loompy
import palettable

sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..', '..')))
from scbeta_scrnaseq import utils
from scbeta_scrnaseq import vis
import scbeta_scrnaseq.cytograph_inmem_utils as cgm
import scbeta_scrnaseq.cytograph_analyses as cga



Populating the interactive namespace from numpy and matplotlib


### Extract cells on the neurog3 trajectory

In [2]:
full_ds = loompy.connect(f'../data/complete_processing/stage5.processed.loom', 'r')
full_ds.vals = sp.sparse.csr_matrix(full_ds.layers[""][:, :])


In [3]:
%mkdir ../data/pseudotime/
attributes_to_copy = ['CellID', 'CellBatch', 'CellDay', 'CellFlask', 'DetailedLabels', 'Labels', '_TrainFilter', '_Valid']

f_clusters = np.isin(full_ds.ca.DetailedLabels, ['scbeta', 'ec', 'neurog3_late', 'neurog3_mid', 'neurog3_early', 'prog_nkx61'])
f_cells = np.where((f_clusters))[0]
min_cells_per_valid_gene = 10

%rm ../data/pseudotime/stage5.scbeta_ec_neurog3.loom
loompy.create(f'../data/pseudotime/stage5.scbeta_ec_neurog3_prog.loom',
              {'': full_ds.vals[:, f_cells]},
              {'Gene': full_ds.ra.Gene,
               '_Valid': (full_ds.vals[:, f_cells].sum(1).A.ravel() > min_cells_per_valid_gene),
               '_Regress': (full_ds.vals[:, f_cells].sum(1).A.ravel() > min_cells_per_valid_gene),
                  },
              {a: full_ds.ca[a][f_cells] for a in attributes_to_copy},
             )
full_ds.close()

In [4]:
import json
with open('../../01_Stages_3_to_6/data/original/cell_cycle_genes.json', 'r') as f:
    cell_cycle_set = set(json.load(f))
     

In [5]:
tds = {}
fn = 'scbeta_ec_neurog3_prog'
tds[fn] = loompy.connect(f'../data/pseudotime/stage5.{fn}.loom')
_tds = tds[fn]

In [6]:
seed = 91829211
tds = {}

for tp in [fn]:
    tds[tp] = loompy.connect(f'../data/pseudotime/stage5.{tp}.loom')
    tds[tp].vals = sp.sparse.csr_matrix(tds[tp].layers[""][:, :])
    _tds = tds[tp]
    
    _valid_genes = _tds.ra._Valid
    _valid_genes[np.where(np.isin(_tds.ra.Gene, list(cell_cycle_set)))[0]] = 0
    _tds.ra._Valid = _valid_genes
    
    cga.highvar_pca(_tds, _tds.vals, namespace='', seed=seed,
                train_cells=np.where(_tds.ca['_TrainFilter'] > 0)[0],
                n_highvar_pcs=50,
                n_highvar_genes=2000)
    
#     tds.ca["HighVarTSNE"] = cg.TSNE(perplexity=100).layout(tds.ca[r"HighVarPCA"][:, :10])

In [ ]:
# seed = 91829211
# for tp in range(2):
#     _tds = tds[tp]
#     _tds.ca["HighVarTSNE"] = cg.TSNE(perplexity=100).layout(_tds.ca[r"HighVarPCA"][:, :10])

### Computation of pseudotime

In [7]:
time_alignment_vector = {}
time_alignment_vector[fn] = pd.Series(tds[fn].ca.CellDay, tds[fn].ca.CellID)

for tp in tds.keys():
    _tds = tds[tp]
    import scanpy.api as sc
    adata = sc.AnnData(_tds.ca.HighVarPCA[:, :10])
    adata.obs_names = _tds.ca.CellID
    adata.obsm['X_pca'] = _tds.ca.HighVarPCA[:, :10]
    sc.pp.neighbors(adata, n_neighbors = 100, n_pcs = adata.obsm['X_pca'].shape[1])
    sc.tl.diffmap(adata)
    
    _tds.ca['DiffMap'] = adata.obsm['X_diffmap']
    
    diffmap1_time_corr = pd.Series(adata.obsm['X_diffmap'][:, 1], _tds.ca.CellID).corr(time_alignment_vector[tp])
    adata.uns['iroot'] = adata.obsm['X_diffmap'][:, 1].argmin() if diffmap1_time_corr>0 else adata.obsm['X_diffmap'][:, 1].argmax()
    sc.tl.dpt(adata, n_branchings=0, n_dcs=10,  min_group_size=1000)
    

In [8]:
grp0 = np.isin(_tds.ca.Labels, ['neurog3_late', 'neurog3_mid', 'neurog3_early', 'prog_nkx61'])
grp1 = np.isin(_tds.ca.Labels, ['scbeta'])
grp2 = np.isin(_tds.ca.Labels, ['ec'])

np.random.seed(1039094)
branch = pd.Series(0, index=adata.obs['dpt_pseudotime'].index)
branch[grp0] = np.random.randint(0,2,sum(grp0))
branch[grp1] = 1
branch[grp2] = 0

pseudotime = pd.Series(0, index=adata.obs['dpt_pseudotime'].index)

_tip_val = 0.50
for b in [0, 1]:
    n_branch_tip = sum(branch==b) - sum((branch == b) & ~(grp0))

    _shared = adata.obs['dpt_pseudotime'][branch==b].rank() <= n_branch_tip
    _tip = ~_shared
    
    _shared = _shared[_shared].index
    _tip = _tip[_tip].index

    pseudotime[_shared] = adata.obs['dpt_pseudotime'][_shared].rank()/len(_shared) * _tip_val
    pseudotime[_tip] = _tip_val + (adata.obs['dpt_pseudotime'][_tip].rank()/len(_tip) * _tip_val)

In [9]:
_tds.ca.PseudotimeBranch = branch[_tds.ca.CellID].values
_tds.ca.PseudotimeRank = pseudotime[_tds.ca.CellID].values

In [13]:
# copy to cluster
_tds.close()
%cp -v ../data/pseudotime/stage5.scbeta_ec_neurog3_prog.loom /Volumes/adrianveres/scbeta_scrnaseq__data/

../data/pseudotime/stage5.scbeta_ec_neurog3_prog.loom -> /Volumes/adrianveres/scbeta_scrnaseq__data/stage5.scbeta_ec_neurog3_prog.loom


In [ ]:
# Execute "python scbeta_scrnaseq/scripts/vgam_regression_wrapper.py scbeta_scrnaseq__data/stage5.scbeta_ec_neurog3_prog.loom"

# sbatch -p test -n 4 -N 1 --job-name ngn3 --mem 12000 --time 06:00:00 \
#         --wrap """source activate py36; module load gcc/7.1.0-fasrc01 R/3.5.0-fasrc02;export R_LIBS_USER=$HOME/apps/R:$R_LIBS_USER;\
#             cd /n/home15/adrianveres; python scbeta_scrnaseq/scripts/vgam_regression_wrapper.py scbeta_scrnaseq__data/stage5.scbeta_ec_neurog3_prog.loom"""
        

In [ ]:
# Copy the file back
# %cp -v /Volumes/adrianveres/scbeta_scrnaseq__data/stage5.scbeta_ec_neurog3_prog.loom  ../data/pseudotime/

In [25]:
_tds.close()

In [1]:
%cp -v /Volumes/adrianveres/scbeta_scrnaseq__data/stage5.scbeta_ec_neurog3_prog.loom ../data/pseudotime/

In [27]:

tds = {}
fn = 'scbeta_ec_neurog3_prog'
tds[fn] = loompy.connect(f'../data/pseudotime/stage5.{fn}.loom')
_tds = tds[fn]

from scbeta_scrnaseq.pseudotime import annotate_vgam_ds
annotate_vgam_ds(_tds)